In [ ]:
# 順張り戦略・逆張り戦略の検討

In [ ]:
from pathlib import Path
import csv
import datetime
import pickle
from itertools import product

import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import tqdm

import data_fetcher
import stock

In [ ]:
def short_trade(df, sell_key, buy_key, sell_price_key, buy_price_key, wall_timestep, slipage=0.0):
    profits = np.zeros(len(df))
    for i in range(len(df)):
        if not df[sell_key][i]:
            continue

        sell_price = df[sell_price_key][i] * (1.0 - slipage)
        buy_price = df["close"][min(len(df) - 1, i + wall_timestep)]
        for j in range(i + 1, min(len(df) - 1, i + wall_timestep)):
            if df[buy_key][j]:
                buy_price = df[buy_price_key][j] * (1.0 + slipage) 
                break
        profits[i] = sell_price / buy_price - 1.0
    return profits

def long_trade(df, sell_key, buy_key, sell_price_key, buy_price_key, wall_timestep, slipage=0.0):
    profits = np.zeros(len(df))
    for i in range(len(df)):
        if not df[buy_key][i]:
            continue

        buy_price = df[buy_price_key][i] * (1.0 + slipage)
        sell_price = df["close"][min(len(df) - 1, i + wall_timestep)]
        for j in range(i + 1, min(len(df) - 1, i + wall_timestep)):
            if df[sell_key][j]:
                sell_price = df[sell_price_key][j] * (1.0 - slipage)
                break
        profits[i] = sell_price / buy_price - 1.0    
    return profits

In [ ]:
def simulate(df, rate, slipage, wall_timestep, pips):
    df = df.with_columns(
        (pl.col("close") + pl.col("ATR") * rate).shift().alias("target_price_high"),
        (pl.col("close") - pl.col("ATR") * rate).shift().alias("target_price_low"),
    ).with_columns(
        ((pl.col("target_price_high") / pips).round() * pips).alias("target_price_high"),
        ((pl.col("target_price_low") / pips).round() * pips).alias("target_price_low"),
    ).with_columns(
        (pl.col("target_price_high") < pl.col("high")).alias("reach_high_target"),
        (pl.col("target_price_low") > pl.col("low")).alias("reach_low_target")
    )    

    # 順張り戦略
    follower_long_profit = long_trade(
        df, 
        sell_key="reach_low_target", 
        buy_key="reach_high_target", 
        sell_price_key="target_price_low",
        buy_price_key="target_price_high",
        slipage=slipage,
        wall_timestep=wall_timestep
    )
    follower_short_profit = short_trade(
        df, 
        sell_key="reach_low_target", 
        buy_key="reach_high_target", 
        sell_price_key="target_price_low",
        buy_price_key="target_price_high",
        slipage=slipage,
        wall_timestep=wall_timestep
    )
    # 逆張り戦略
    contrarian_long_profit = long_trade(
        df, 
        sell_key="reach_high_target", 
        buy_key="reach_low_target", 
        sell_price_key="target_price_high",
        buy_price_key="target_price_low",
        slipage=slipage,
        wall_timestep=wall_timestep
    )
    contrarian_short_profit = short_trade(
        df, 
        sell_key="reach_high_target", 
        buy_key="reach_low_target", 
        sell_price_key="target_price_high",
        buy_price_key="target_price_low",
        slipage=slipage,
        wall_timestep=wall_timestep
    )
    return follower_long_profit, follower_short_profit, contrarian_long_profit, contrarian_short_profit

In [ ]:
# slipageの計算
# tickの値動きの差分の標準偏差から求める
def calc_average_slipage():
    tick_df = data_fetcher.gmo.GMOFethcer().fetch_ticker("BTC_JPY")
    prices = tick_df["price"].to_numpy()
    diff = np.abs(prices[:-1] - prices[1:]) / prices[:-1]
    slip_mean = diff.mean()
    slip_std = diff.std()
    print(slip_mean, slip_std)
    return slip_mean, slip_std

slip_mean, slip_std = calc_average_slipage()

In [ ]:
def run(output_path = None, start_date=datetime.datetime(2024, 1, 1), end_date=datetime.datetime(2024, 1, 31)):
    symbols = ["BTC_JPY"]
    intervals = [1, 5, 15, 60, 180]
    slipages = [0, slip_mean, slip_mean + slip_std]
    rates = [0.1, 0.2, 0.5, 0.7, 1.0]
    wall_timestep = 100
    pips = 1

    results = []
    for symbol, interval in product(symbols, intervals):
        fetcher = data_fetcher.gmo.GMOFethcer()
        df = fetcher.fetch_ohlc(symbol, interval=datetime.timedelta(minutes=interval), start_date=start_date, end_date=end_date)
        df = stock.crypto.feature.calc_features(df).filter(
            pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
        )
        for slipage, rate in product(slipages, rates):
            flp, fsp, clp, csp = simulate(df, rate=rate, slipage=slipage, wall_timestep=wall_timestep, pips=pips)
            # print("symbol = {}, interval = {}, slipage = {}, rate = {}, follower long = {}, follower short = {}, cont long = {}, cont short = {}".format(
            #     symbol, interval, slipage, rate, flp.sum(), fsp.sum(), clp.sum(), csp.sum()
            # ))
            results.append([symbol, interval, slipage, rate, flp, fsp, clp, csp])

    if output_path is not None:
        header = ["symbol", "interval", "slipage", "rate"] + sum([
            [f"{prefix}_profit", f"{prefix}_num_tick", f"{prefix}_est_num_trade", f"{prefix}_avg_profit_per_trade", f"{prefix}_profit_std", f"{prefix}_max_drawdown"] 
            for prefix in ["follower_long", "follower_short", "contrarian_long", "contrarian_short"]], [])
        rows = []
        for res in results:
            row = res[:4]
            for profit_arr in res[4:]:
                profit = profit_arr.sum()
                num_tick = len(profit_arr)
                est_num_trade = (np.abs(profit_arr) >  1e-9).sum()
                avg_profit_per_trade = profit / est_num_trade
                profit_std = profit_arr[np.abs(profit_arr) > 1e-9].std()
                max_drawdown = profit_arr.min()
                row += [profit, num_tick, est_num_trade, avg_profit_per_trade, profit_std, max_drawdown]
            rows.append(row)
                
        with open(output_path, "w") as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(header)
            csv_writer.writerows(rows)

    return results

start_date = datetime.datetime(2021, 9, 5)
delta = datetime.timedelta(days=90)
outputs = []
while start_date < datetime.datetime.now() - delta:
    print("start date = {}".format(start_date))
    end_date = start_date + delta
    output_path = "./test_{}_{}.csv".format(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"))
    outputs.append(output_path)
    results = run(output_path=output_path, start_date=start_date, end_date=end_date)
    start_date += delta

In [ ]:
dfs = [pl.read_csv(path) for path in outputs]

In [ ]:
df = pl.concat([df.with_columns(pl.lit(Path(outputs[idx]).stem).alias("stem")) for idx, df in enumerate(dfs)])

In [ ]:
# 結果の比較

In [ ]:
# 順張り戦略と逆張り戦略の比較
def plot(df, intervals=[1], rates=[0.1, 0.2, 0.5, 0.7, 1.0], slipages=[0.0, 0.0000838, 0.00031], cont_keys=["contrarian_long_profit"], follow_keys=["follower_long_profit"]):
    flp = {}
    clp = {}
    for interval in intervals:
        for rate in rates:
            filtered = df.filter((pl.col("interval") == interval) & ((pl.col("rate") - rate).abs() < 1e-5) & (pl.col("slipage") < 1e-5))
            for key in cont_keys:
                clp[f"{key}_{interval}_{rate}"] = filtered[key]

            for slipage in slipages:
                filtered = df.filter((pl.col("interval") == interval) & ((pl.col("rate") - rate).abs() < 1e-5) & ((pl.col("slipage") - slipage).abs() < 1e-5))
                for key in follow_keys:
                    flp[f"{key}_{interval}_{rate}_{slipage}"] = filtered[key]

    for key, val in flp.items():
        plt.plot(val, label=key)
    for key, val in clp.items():
        plt.plot(val, label=key)

    plt.legend()
    plt.show()

In [ ]:
# 1分足
plot(df, intervals=[1], rates=[0.5], slipages=[0.0000838, 0.00031])
plot(df, intervals=[1], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
# 5分足
plot(df, intervals=[5], rates=[0.5], slipages=[0.0000838, 0.00031])
#plot(df, interval=[5], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
# 15分足
plot(df, intervals=[15], rates=[0.5], slipages=[0.0000838, 0.00031])
#plot(df, interval=[5], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
# 60分足
plot(df, intervals=[60], rates=[0.5], slipages=[0.0000838, 0.00031])
#plot(df, interval=[5], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
# 180分足
plot(df, intervals=[180], rates=[0.5], slipages=[0.0000838, 0.00031])
#plot(df, interval=[5], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
# 60分足
plot(df, intervals=[60], slipages=[0.00031], cont_keys=[])
#plot(df, interval=[5], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
# 60分足
plot(df, intervals=[1, 5, 15, 60, 180], rates=[0.7], slipages=[0.00031], cont_keys=[])
#plot(df, interval=[5], rates=[0.2], slipages=[0.0000838, 0.00031])

In [ ]:
dfs[0].filter(pl.col("slipage") > 1e-4)

In [ ]:
plt.plot(buy_profits.cumsum(), label="buy")
plt.plot(sell_reverse_profits.cumsum(), label="sell_reverse")
plt.plot(sell_profits.cumsum(), label="sell")
plt.plot(buy_reverse_profits.cumsum(), label="buy_reverse")
plt.plot(df["close"] / df["close"][0] - 1.0, label="close")
plt.legend()